## Initialization Notebook

#### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class Task:
    def __init__(self, x, y):
        self.name = "Task"
        self.x = x
        self.y = y
        self.reward = np.random.randint(0, 100)

    def __str__(self):
        return f"Task at ({self.x}, {self.y})"

    def __repr__(self):
        return f"Task at ({self.x}, {self.y}) with reward {self.reward}"
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

In [ ]:
# Create a list of tasks
tasks = [Task(np.random.randint(0, 10), np.random.randint(0, 10)) for _ in range(10)]

# Extract x, y, and rewards
x_vals = [task.x for task in tasks]
y_vals = [task.y for task in tasks]
rewards = [task.reward for task in tasks]

# Create a grid for the heatmap
grid_size = (max(x_vals) + 1, max(y_vals) + 1)
reward_grid = np.zeros(grid_size)

# Populate the grid with rewards
for task in tasks:
    reward_grid[task.x, task.y] = task.reward

# Create a mask for cells where the reward is 0
mask = reward_grid == 0

# Plot the heatmap with a mask and without text for zero-valued cells
plt.figure(figsize=(8, 6))
sns.heatmap(reward_grid, mask=mask, annot=True, fmt=".0f", cmap="coolwarm", 
            linewidths=0.5, square=True, cbar_kws={"label": "Reward"}, 
            annot_kws={"color": "black"})

# Set color of masked cells to white (or transparent if using a more advanced library)
plt.title("Task Rewards Heatmap")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()